In [1]:
import pandas as pd
import re

In [2]:
# load data
filepath = '/home/silas/Documents/DataScience Notebooks/scraping/WikiNews/scrape_wikinews.csv'
df = pd.read_csv(filepath,
                names = ['id','title','url','date_written','keywords','content'],
                dtype={'id':'str'})

In [3]:
#str -> list
df['keywords'] =df['keywords'].apply(lambda x: re.findall(r"'([^']+)'",x))

In [4]:
def make_dicts():
    
    def makeDateDict():
        date_lst = list(df['date_written'].unique())
        date_lst.remove('None')
        return {elm:idx for idx,elm in enumerate(date_lst)} 
    
    def makeKeywordDict():
        st = set()
        for elm in df['keywords']:
            if elm != None: st.update(elm)
        return {elm:idx for idx,elm in enumerate(st)}

    return makeDateDict(), makeKeywordDict(),

#get dicts
date_dict, keyword_dict = make_dicts()

In [5]:
# article csv
def make_article_csv():
    article_df = pd.DataFrame(
        data={'id':list(df.id),
              'time_id':list(map(lambda x: str(date_dict[x]) if x != "None" else None,list(df.date_written)))
             }
                            )
    article_df = article_df.join(df[['id','url','content','title']].set_index('id'),on='id')
    article_df.to_csv('wiki_article.csv',index=False)

In [6]:
def make_simple_csv(dictx,filename):
    frame = pd.DataFrame({'id':list(dictx.values()), 'value':list(dictx)})
    frame.drop_duplicates().to_csv(filename,index=False)

In [7]:
def make_multi_csv(dictx,filename,column_name,id_name):
    frame = pd.DataFrame(df[['id',column_name]])
    frame = frame.dropna()
    frame = frame.explode(column_name)
    frame[column_name] = frame[column_name].apply(lambda x: dictx[x])
    frame.drop_duplicates().to_csv(filename, index=False, header=['id',id_name])

In [8]:
make_article_csv()

make_simple_csv(date_dict,'wiki_time.csv')
make_simple_csv(keyword_dict,'wiki_keyword.csv')

make_multi_csv(keyword_dict,'wiki_keywords.csv','keywords','keyword_id')